In [1]:
import sys

import os
import argparse
import pandas as pd
import numpy as np
import torch
sys.path.append(r"../")
sys.path.append(r"../../")
sys.path.append('/home/wyliu/code/CB-IV')
from utils import log, CausalDataset
from module.SynCBIV import run as run_SynCBIV

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--ood',default=0,type=float,help='The train dataset of OOD')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=False,type=bool,help='The use of GPU')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=10,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mXs',default=2,type=int,help='The dim of Noise variables D')
    argparser.add_argument('--storage_path',default='../../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

In [2]:
''' bias rate '''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 1.3, 1.5, 2.0, 2.5, 3.0, 0.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}
which_benchmark = 'SynOOD_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}_{args.mode}/ood{brdc[args.ood]}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'

In [3]:
exp = 0
train_df = pd.read_csv(dataDir + f'{exp}/ood_{brdc[args.ood]}/{args.mode}/train.csv')

In [4]:

train_df

,v0,v1,u0,u1,u2,u3,x0,x1,x2,x3,...,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13
0,0.493977,-0.260800,-0.711891,-0.292161,1.040120,0.150134,-0.170383,-0.440823,-0.768965,-0.409060,...,-0.768965,-0.409060,0.434796,-1.495194,-0.391358,-0.738139,-1.815272,-0.401515,-0.197004,-0.295830
1,0.336289,-0.363355,-1.943483,0.844260,1.589132,-0.526447,-1.036420,0.753754,0.863356,0.567789,...,0.863356,0.567789,1.156842,-0.776830,1.106095,0.400007,1.906140,-1.103625,-1.308841,-1.040354
2,0.492186,0.174818,0.099783,-0.238490,-0.665089,0.961982,0.097927,1.012746,-0.159527,-0.322541,...,-0.159527,-0.322541,-1.084459,0.034715,0.512465,0.080086,-1.864322,0.268274,0.230852,-0.417562
3,-0.470176,-0.459523,-0.046204,-1.076059,-2.348329,-0.250041,0.639481,1.028225,0.621134,2.143710,...,0.621134,2.143710,-0.288671,-0.081083,-1.100671,1.151920,1.854969,0.350531,-2.947761,0.499064
4,-0.130730,0.342468,0.168469,-0.533123,2.417915,0.076324,1.239516,1.978109,-0.416655,0.332296,...,-0.416655,0.332296,-1.206405,-0.850588,0.139575,0.045569,1.182868,0.198049,-0.879902,0.111329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.575986,1.812700,-0.435999,-2.080857,-0.036692,1.581702,0.638811,0.739790,-0.798585,0.066193,...,-0.798585,0.066193,-0.854370,-0.605160,-0.196696,-0.832622,-2.977858,-1.473536,0.148308,0.925429
9996,0.265083,0.996480,1.633413,0.116476,0.476131,-1.138077,0.160802,0.379243,0.402359,-0.929106,...,0.402359,-0.929106,1.102149,-1.074471,-0.085730,2.017610,-0.036406,0.446000,-0.077953,0.745038
9997,0.933596,0.565541,-0.850877,-0.800897,0.713363,0.745454,1.948265,0.462898,-0.952901,0.890370,...,-0.952901,0.890370,0.319111,-0.505228,2.217786,0.367356,-0.131835,-0.598144,-1.435421,-1.209461
9998,-0.718961,-0.376158,0.753122,-0.199558,-2.501154,-0.610095,-1.829971,-0.486355,-0.689859,-1.856230,...,-0.689859,-1.856230,0.339620,-0.415959,-1.524185,-0.558562,-0.193542,0.719911,-0.675151,-0.823283
